In [12]:
from pyspark import SparkContext, SparkConf 
import pandas as pd
import re
from operator import add
import numpy as np
import os
os.environ["PYSPARK_PYTHON"]="/anaconda3/envs/tensorflow/bin/python3.5"
os.environ["PYTHONHASHSEED"]="123"
#data1 = pd.read_csv("_Answers.csv",encoding="latin-1", iterator = True, chunksize=100)

sc = SparkContext.getOrCreate()

questionFile=sc.textFile("test_q.csv")
answerFile=sc.textFile("test_a.csv")
tagFile = sc.textFile("test_t.csv")
stopWordsFile = sc.textFile("stopwords.txt")
print("finished")

questionWithHeader = questionFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = questionWithHeader.first() #extract header
question = questionWithHeader.filter(lambda x: x != header)
print(question.count())

questionsample=question.sample(False,0.05,81)
questionLower = questionsample.map(lambda x: (x[0], (x[5]+x[6]).lower()))
def f(x): return re.sub('[^a-zA-Z]+', ' ', x)
questionLower = questionLower.mapValues(f)
print(questionLower.take(2))
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
print(questionPairRaw.take(2))

finished
9999
[('3470', 'how do i transform sql columns into rows p i have a very simple problem which requires a very quick and simple solution in sql server p p i have a table with x columns i want to be able to select one row from the table and then transform the columns into rows p pre code tablea column column column code pre p sql statement to ruturn p pre code resulta value of column value of column value of column code pre hr p strong kevin strong i ve had a google search on the topic but alot of the example where overly complex for my example strong are you able to help further strong p p mario the solution i am creating has columns which stores the values to and i must work out how many columns have the value or more so i thought about creating a query to turn that into rows and then using the generated table in a subquery to say count the number of rows with column p '), ('4320', 'asp net application without business logic layer p is it acceptable to have an strong asp net s

In [ ]:
#Clustring
from pyspark.sql import SQLContext, Row
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
import itertools
from collections import defaultdict
from pyspark.mllib.linalg import Vectors
import os

os.environ['PYSPARK_PYTHON']="/anaconda3/envs/tensorflow/bin/python3.5"

sqlContext = SQLContext(sc)

#Constant
num_of_stop_words=50
num_topics=3

def document_vector(document):
    id = document[1]
    counts = defaultdict(int)
    for token in document[0]:
        if token in vocabulary:
            token_id = vocabulary[token]
            counts[token_id] += 1
    counts = sorted(counts.items())
    keys = [x[0] for x in counts]
    values = [x[1] for x in counts]
    return (id, Vectors.sparse(len(vocabulary), keys, values))


data=questionPairRaw
data = data.map(lambda words: words[1])#.filter(lambda x: len(x)>3)  
data = data.map(lambda words: (words,1))
data = data.reduceByKey(add) 
data=data.map(lambda tuple: (tuple[1], tuple[0])).sortByKey(False)

threshold=data.take(num_of_stop_words)[num_of_stop_words - 1][0]

vocabulary = data.filter(lambda x: x[0] < threshold)
vocabulary = vocabulary.map(lambda x:x[1]).zipWithIndex().collectAsMap()

inv_voc = {value: key for (key, value) in vocabulary.items()}

documents=questionLower#.map(lambda doc:[x for x in doc if len(x[1])>3])
documents=documents.map(lambda tuple: (tuple[1], tuple[0]))

documents=documents.zipWithIndex()
documents=documents.map(document_vector)
documents=documents.map(list)
print(documents.take(10))
#print(vocabulary)
#all docs



with open("output.txt", 'w') as f:
    lda_model = LDA.train(documents, k=num_topics, maxIterations=100)
                    
    topic_indices = lda_model.describeTopics(maxTermsPerTopic=100)
    
    # Output topics. Each is a distribution over words (matching word count vectors)
    print("Learned topics (as distributions over vocab of " + str(lda_model.vocabSize())\
      + " words):")
    topics = lda_model.topicsMatrix()
    for topic in range(3):
        print("Topic " + str(topic) + ":")
    for word in range(0, lda_model.vocabSize()):
        print(" " + str(topics[word][topic]))

        
    # Print topics, showing the top-weighted 10 terms for each topic
    for i in range(len(topic_indices)):
        f.write("Topic #{0}\n".format(i + 1))
        for j in range(len(topic_indices[i][0])):
            f.write("{0}\t{1}\n".format(inv_voc[topic_indices[i][0][j]] \
                .encode('utf-8'), topic_indices[i][1][j]))
            

    f.write("{0} topics distributed over {1} documents and {2} unique words\n"  \
        .format(num_topics, documents.count(), len(vocabulary)))
    


In [6]:
answerFileWithHeader = answerFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)

header = answerFileWithHeader.first() 
answer = answerFileWithHeader.filter(lambda x: x != header)
print(answer.take(2))

answer=answer.map(lambda x: (x[0],x[5]))
print(answer.take(2))

def f(x): return re.sub('[^a-zA-Z]+', ' ', x)
answerLower = answer.mapValues(f)
print(answerLower.take(2))

answerwithTag = answerLower.map(lambda x:(x[0],"Answer@"+x[0]+x[1]))
print(answerwithTag.take(2))

#(questionid, sum of answers)
def sumAsList(x):
    list=[]
    list.append(x[0])
    list.append(x[1])
    return list
answerSum=answerwithTag.reduceByKey(sumAsList)
print(answerSum.take(2))

[['92', '61', '2008-08-01T14:45:37Z', '90', '13', '"<p><a href=""http://svnbook.red-bean.com/"">Version Control with Subversion</a></p>    <p>A very good resource for source control in general. Not really TortoiseSVN specific; though.</p>"', 'False'], ['124', '26', '2008-08-01T16:09:47Z', '80', '12', '"<p>I wound up using this. It is a kind of a hack; but it actually works pretty well. The only thing is you have to be very careful with your semicolons. : D</p>  <pre><code>var strSql:String = stream.readUTFBytes(stream.bytesAvailable);       var i:Number = 0; var strSqlSplit:Array = strSql.split("";""); for (i = 0; i &lt; strSqlSplit.length; i++){     NonQuery(strSqlSplit[i].toString()); } </code></pre> "', 'True']]
[('92', '"<p><a href=""http://svnbook.red-bean.com/"">Version Control with Subversion</a></p>    <p>A very good resource for source control in general. Not really TortoiseSVN specific; though.</p>"'), ('124', '"<p>I wound up using this. It is a kind of a hack; but it actuall